<a href="https://colab.research.google.com/github/GuyWhoCodesThings/Custom-Functions-for-Scratch-Neural-Network/blob/main/NN%20with%20new%20Layers%20and%20Functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy import signal
from keras.datasets import mnist
from keras.utils import np_utils
from skimage.measure import block_reduce
from skimage.measure import block_reduce
from skimage.util import view_as_blocks
import numpy.ma as ma
from matplotlib import pyplot
import random

In [ ]:
!python --version

Python 3.7.14


In [ ]:
class Layer:
  def __init__(self):
    self.input = None
    self.output = None
  def forward(self, input):
    pass
  def backward(self, output_gradient, alpha):
    pass

In [ ]:
class Dense(Layer):
    def __init__(self, input_size, output_size):
        self.weights = np.random.randn(output_size, input_size)
        self.bias = np.random.randn(output_size, 1)
    def forward(self, input):
        self.input = input
        return np.dot(self.weights, self.input) + self.bias
    def backward(self, output_gradient, learning_rate):
        weights_gradient = np.dot(output_gradient, self.input.T)
        self.weights -= learning_rate * weights_gradient
        self.bias -= learning_rate * output_gradient
        return np.dot(self.weights.T, output_gradient)

In [ ]:
class Convolutional(Layer):
     def __init__(self, input_shape, kernel_size, depth):
         input_depth, input_height, input_width = input_shape
         self.depth = depth
         self.input_shape = input_shape
         self.input_depth = input_depth
         self.output_shape = (depth, input_height - kernel_size + 1, input_width - kernel_size + 1)
         self.kernels_shape = (depth, input_depth, kernel_size, kernel_size)
         self.kernels = np.random.randn(*self.kernels_shape)
         self.biases = np.random.randn(*self.output_shape)

     def forward(self, input):
         self.input = input
         self.output = np.copy(self.biases)
         for i in range(self.depth):
             for j in range(self.input_depth):
                 self.output[i] += signal.correlate2d(self.input[j], self.kernels[i, j], "valid")
         return self.output

     def backward(self, output_gradient, learning_rate):
         kernels_gradient = np.zeros(self.kernels_shape)
         input_gradient = np.zeros(self.input_shape)

         for i in range(self.depth):
             for j in range(self.input_depth):
                 kernels_gradient[i, j] = signal.correlate2d(self.input[j], output_gradient[i], "valid")
                 input_gradient[j] += signal.convolve2d(output_gradient[i], self.kernels[i, j], "full")

         self.kernels -= learning_rate * kernels_gradient
         self.biases -= learning_rate * output_gradient
         return input_gradient

In [ ]:
def get_ordered_blocks(input, depth, block_size, input_shape):
    ordered_blocks = input.reshape(depth, -1, block_size, block_size)
    reshaped_arr = np.zeros(input_shape)
    for i in range(depth):
      split_blocks = np.array(np.hsplit(ordered_blocks[i], block_size))
      #splits array of blocks evenly with respect to the original image length and block size
      split_blocks = np.array(np.hsplit(split_blocks, 12))
      #reshape array to get it into a sqaure shape like orignal image matrix
      reshaped_arr[i] = split_blocks.flatten().reshape(1, input_shape[2], input_shape[1])
    return reshaped_arr

In [ ]:
def expand_output(output_gradient, depth, height, width, block_size, output_height, output_width):
  output_gradient = output_gradient.reshape(depth, output_height, output_width)
  input_grad = np.zeros((depth, int(output_height*output_height), block_size**2))
  for i in range(depth):
    input_grad[i] = np.repeat(output_gradient[i], block_size**2).reshape(-1,block_size**2)
  return get_ordered_blocks(input_grad, depth, block_size, (depth, height, width))


In [ ]:
class MaxPool2d(Layer):
  def __init__(self, input_shape, block_size):
    self.block_size = block_size
    self.input_shape = input_shape
    input_depth, input_height, input_width = input_shape
    self.depth = input_depth
    self.input_height = input_height
    self.input_width = input_width
    self.output_width = int(self.input_width / self.block_size)
    self.output_height = int(self.input_height / self.block_size)
    
  def forward(self, input):
    self.block_image = view_as_blocks(input, (1,self.block_size, self.block_size)).reshape(self.depth,-1, self.block_size**2)
    self.output = np.amax(self.block_image, axis=2).reshape(-1, self.output_height, self.output_width)
    return self.output
    
  def backward(self, output_gradient, learning_rate):
    max_indices = np.argmax(self.block_image, axis=2)
    max_indices = max_indices.reshape(self.depth, -1, 1)
    input_gradient = np.zeros_like(self.block_image).reshape(self.depth, -1, self.block_size**2)
    np.put_along_axis(input_gradient, max_indices, 1, axis=2)
    return np.multiply(get_ordered_blocks(input_gradient, self.depth, self.block_size, self.input_shape),
                      expand_output(output_gradient, self.depth, self.input_height, self.input_width,
                      self.block_size, self.output_width, self.output_height))

In [ ]:
class Reshape(Layer):
     def __init__(self, input_shape, output_shape):
         self.input_shape = input_shape
         self.output_shape = output_shape

     def forward(self, input):
         return np.reshape(input, self.output_shape)

     def backward(self, output_gradient, learning_rate):
         return np.reshape(output_gradient, self.input_shape)

In [ ]:
def binary_cross_entropy(y_true, y_pred):
     return np.mean(-y_true * np.log(y_pred) - (1 - y_true) * np.log(1 - y_pred))

def binary_cross_entropy_prime(y_true, y_pred):
     return ((1 - y_true) / (1 - y_pred) - y_true / y_pred) / np.size(y_true)

In [ ]:
class Softmax(Layer):

  def forward(self, input):
    tmp = np.exp(input)
    self.output = tmp / np.sum(tmp)
    return self.output
  
  def bacward(self, output_gradient, alpha):
    n = np.size(self.output)
    tmp = np.tile(self.output, n)
    return np.dot(tmp * (np.identity(n) - np.transpose(tmp)), output_gradient)


In [ ]:
class Activation(Layer):
    def __init__(self, activation, activation_prime):
        self.activation = activation
        self.activation_prime = activation_prime
    def forward(self, input):
        self.input = input
        return self.activation(self.input)
    def backward(self, output_gradient, learning_rate):
        return np.multiply(output_gradient, self.activation_prime(self.input))

In [ ]:
class ReLU(Activation):

  def __init__(self):

    relu = lambda x: np.maximum(x,0)
    relu_prime = lambda x: x > 0
    
    super().__init__(relu, relu_prime)

In [ ]:
class Soft_Plus(Activation):

  def __init__(self):

    softPlus = lambda x: np.log1p(np.exp(x))
    soft_prime = lambda x: 1 / (1 + np.exp(-x))

    super().__init__(softPlus, soft_prime)

In [ ]:
class Tanh(Activation):

  def __init__(self):

    tanh = lambda x: np.tanh(x)
    tanh_prime = lambda x: 1 - np.tanh(x) ** 2
    
    super().__init__(tanh, tanh_prime)

In [ ]:
class Sigmoid(Activation):
     def __init__(self):
         def sigmoid(x):
            return 1 / (1 + np.exp(-x))

         def sigmoid_prime(x):
            s = sigmoid(x)
            return s * (1 - s)

         super().__init__(sigmoid, sigmoid_prime)

In [ ]:
def mse(y_true, y_pred):
  return np.mean(np.power(y_true - y_pred, 2))

def mse_prime(y_true, y_pred):
  return 2 * (y_pred - y_true) / np.size(y_true)

In [ ]:
def preprocess_data(x, y, limit):
     zero_index = np.where(y == 0)[0][:limit]
     one_index = np.where(y == 1)[0][:limit]
     all_indices = np.hstack((zero_index, one_index))
     all_indices = np.random.permutation(all_indices)
     x, y = x[all_indices], y[all_indices]
     x = x.reshape(len(x), 1, 28, 28)
     x = x.astype("float32") / 255
     y = np_utils.to_categorical(y)
     y = y.reshape(len(y), 2, 1)
     return x, y

 # load MNIST from server, limit to 100 images per class since we're not training on GPU
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, y_train = preprocess_data(x_train, y_train, 100)
x_test, y_test = preprocess_data(x_test, y_test, 100)

 # neural network
network = [
     Convolutional((1, 28, 28), 5, 5),
     Sigmoid(),
     Reshape((5, 24, 24), (5 * 24 * 24, 1)),
     Dense(5 * 24 * 24, 100),
     Sigmoid(),
     Dense(100, 2),
     Sigmoid()
 ]

epochs = 50
learning_rate = 0.01

 # train
for e in range(epochs):
     error = 0
     for x, y in zip(x_train, y_train):
         # forward
         output = x
         for layer in network:
             output = layer.forward(output)

         # error
         error += binary_cross_entropy(y, output)

         # backward
         grad = binary_cross_entropy_prime(y, output)
         for layer in reversed(network):
             grad = layer.backward(grad, learning_rate)

     error /= len(x_train)
     print(f"{e + 1}/{epochs}, error={error}")

 # test
for x, y in zip(x_test, y_test):
     output = x
     for layer in network:
         output = layer.forward(output)
     print(f"pred: {np.argmax(output)}, true: {np.argmax(y)}")

1/50, error=0.7876186329301654
2/50, error=0.1508600299824121
3/50, error=0.09105359821136086
4/50, error=0.061621558502756836
5/50, error=0.04669033373639828
6/50, error=0.03906706747051978
7/50, error=0.03357446437670198
8/50, error=0.02953237343625992
9/50, error=0.02640176172807907
10/50, error=0.023869579223909452
11/50, error=0.02177742371405563
12/50, error=0.020038611366085538
13/50, error=0.01856809276647478
14/50, error=0.01729608991353508
15/50, error=0.016198721151056708
16/50, error=0.015263855568603784
17/50, error=0.014462265145159853
18/50, error=0.013762041374186615
19/50, error=0.01313977719699551
20/50, error=0.012579903886808241
21/50, error=0.012071752433187899
22/50, error=0.011607535078158017
23/50, error=0.011181225629160041
24/50, error=0.010787965033850166
25/50, error=0.010423742900133634
26/50, error=0.010085212522731398
27/50, error=0.009769565667771051
28/50, error=0.009474434529551094
29/50, error=0.009197809173884207
30/50, error=0.008937967314675017
31/

In [ ]:
network2 = [
     Convolutional((1, 28, 28), 5, 5),
     Sigmoid(),
     MaxPool2d((5, 24, 24), 2),
     Sigmoid(),
     Reshape((5, 12, 12), (5 * 12 * 12, 1)),
     Dense(5 * 12 * 12, 100),
     Sigmoid(),
     Dense(100, 2),
     Sigmoid()
 ]

epochs = 50
learning_rate = 0.01

 # train
for e in range(epochs):
     error = 0
     for x, y in zip(x_train, y_train):
         # forward
         output = x
         for layer in network2:
             output = layer.forward(output)

         # error
         error += binary_cross_entropy(y, output)

         # backward
         grad = binary_cross_entropy_prime(y, output)
         for layer in reversed(network2):
             grad = layer.backward(grad, learning_rate)

     error /= len(x_train)
     print(f"{e + 1}/{epochs}, error={error}")

 #test
for x, y in zip(x_test, y_test):
     output = x
     for layer in network2:
         output = layer.forward(output)
     print(f"pred: {np.argmax(output)}, true: {np.argmax(y)}")

1/50, error=0.8656302932198544
2/50, error=0.541484176828193
3/50, error=0.47287464386228595
4/50, error=0.4178869139115344
5/50, error=0.3753299338758027
6/50, error=0.33883475674536706
7/50, error=0.30686266899142867
8/50, error=0.2787616162820051
9/50, error=0.2539948244880112
10/50, error=0.23221956439253333
11/50, error=0.2129845298717967
12/50, error=0.19586128542117656
13/50, error=0.18060445837440986
14/50, error=0.16721707641914083
15/50, error=0.15559664572103588
16/50, error=0.1453774880860192
17/50, error=0.13629395880953946
18/50, error=0.12815780078604258
19/50, error=0.12082819464403206
20/50, error=0.11420591451622367
21/50, error=0.1081940719954728
22/50, error=0.10269686727740032
23/50, error=0.09762916441292305
24/50, error=0.09288883492124313
25/50, error=0.08827779083560547
26/50, error=0.08342240469266293
27/50, error=0.07838417703716756
28/50, error=0.07443152600202327
29/50, error=0.0711979641890313
30/50, error=0.06826277821621493
31/50, error=0.065549192249258